# **Evaluación Comprehensiva - Metodología Completa**

Sistema de evaluación robusto con:
- Benchmarks simples y complejos
- Múltiples heurísticas de comparación
- Análisis estadístico riguroso
- Visualizaciones comparativas

In [3]:
%run librerias.ipynb

Using device: cpu
Librerías cargadas exitosamente


In [4]:
%run benchmark_loader.ipynb

Using device: cpu
Librerías cargadas exitosamente
CARGANDO SUITE DE BENCHMARKS

Total de benchmarks cargados: 7

          nombre  nodos  aristas  densidad  grado_promedio  grado_max  grado_min  componentes  clustering  cota_superior  chi_teorico
        Petersen     10       15  0.333333        3.000000          3          3            1    0.000000              4          NaN
       Cycle_C20     20       20  0.105263        2.000000          2          2            1    0.000000              3          2.0
       Wheel_W15     15       28  0.266667        3.733333         14          3            1    0.632479             15          NaN
    Complete_K10     10       45  1.000000        9.000000          9          9            1    1.000000             10          NaN
Bipartite_K10_10     20      100  0.526316       10.000000         10         10            1    0.000000             11          2.0
        Tree_n30     30       29  0.066667        1.933333          4          1   

In [5]:
%run gnn_model.ipynb

Using device: cpu
Librerías cargadas exitosamente
Modelo GNN inicializado:
GNNOrdering(
  (convs): ModuleList(
    (0): GCNConv(1, 32)
    (1): GCNConv(32, 32)
  )
  (lin): Linear(in_features=32, out_features=1, bias=True)
)


In [6]:
%run heuristics.ipynb

Using device: cpu
Librerías cargadas exitosamente
Using device: cpu
Librerías cargadas exitosamente
Estadísticas del grafo:
{'nodos': 42, 'aristas': 77, 'densidad': 0.08943089430894309, 'grado_max': 9}

Shape de features: (42, 3)
Shape de edge_index: (2, 77)

Grafo recibido: 42 nodos, 77 aristas
Densidad: 0.08943089430894309, Grado máximo: 9

RESULTADOS DE HEURÍSTICAS
         metodo  colores_promedio  colores_std  colores_min  tiempo_promedio  tiempo_std
         Random               4.2     0.600000            4         0.000030    0.000007
     Grado Desc               4.0     0.000000            4         0.000034    0.000004
   Welsh-Powell               4.0     0.000000            4         0.000037    0.000004
         DSATUR               3.0     0.000000            3         0.000719    0.000028
Parallel Greedy               4.4     0.489898            4         0.000181    0.000009

Resultados del grafo : 42 nodos, 77 aristas
Densidad: 0.08943089430894309, Grado máximo: 9


## **1. PREPARACIÓN DE DATOS PARA GNN**

In [ ]:
def preparar_datos_gnn(G):
    """
    Convierte un grafo NetworkX a formato PyTorch Geometric.
    """
    X = extraer_features(G)
    X = normalizar_features(X)

    edge_index = np.array(list(G.edges())).T

    if edge_index.size == 0:
        edge_index = np.array([[], []], dtype=np.int64)

    edge_index_reverse = np.array([edge_index[1], edge_index[0]])
    edge_index_bidirectional = np.concatenate([edge_index, edge_index_reverse], axis=1)

    x_tensor = torch.FloatTensor(X)
    edge_index_tensor = torch.LongTensor(edge_index_bidirectional)

    from torch_geometric.data import Data
    data = Data(
        x=x_tensor,
        edge_index=edge_index_tensor,
        num_nodes=G.number_of_nodes()
    )

    return data

## **2. FUNCIÓN DE COLORACIÓN GREEDY PARA GNN**

In [8]:
def greedy_coloring_gnn(edge_index, num_nodes, ordering):
    """
    Algoritmo greedy de coloración dado un ordenamiento.
    Compatible con edge_index de PyTorch Geometric.
    """
    if isinstance(edge_index, torch.Tensor):
        edge_index = edge_index.cpu().numpy()
    
    adj = {i: set() for i in range(num_nodes)}
    for i in range(edge_index.shape[1]):
        u, v = edge_index[0, i], edge_index[1, i]
        if u != v:
            adj[u].add(v)
            adj[v].add(u)
    
    coloring = {}
    for node in ordering:
        neighbor_colors = {coloring[v] for v in adj[node] if v in coloring}
        
        color = 0
        while color in neighbor_colors:
            color += 1
        coloring[node] = color
    
    return max(coloring.values()) + 1 if coloring else 0

## **3. EVALUADOR MULTI-HEURÍSTICA**

In [9]:
def evaluar_todas_heuristicas(G, model=None, repeticiones=5):
    """
    Evalúa múltiples heurísticas en un grafo.
    
    Heurísticas evaluadas:
    1. Random (baseline)
    2. Greedy Natural Order
    3. Largest Degree First
    4. Welsh-Powell
    5. DSATUR
    6. GNN-guided (si se proporciona modelo)
    """
    resultados = []
    
    for _ in range(repeticiones):
        order_random = ordenamiento_aleatorio(G)
        t0 = time.time()
        coloring_random = greedy_coloring(G, order_random)
        t1 = time.time()
        metrics_random = evaluar_coloracion(G, coloring_random)
        resultados.append({
            'metodo': 'Random',
            'colores': metrics_random['num_colores'],
            'tiempo': t1 - t0,
            'valido': metrics_random['valido']
        })
    
    order_natural = list(G.nodes())
    t0 = time.time()
    coloring_natural = greedy_coloring(G, order_natural)
    t1 = time.time()
    metrics_natural = evaluar_coloracion(G, coloring_natural)
    resultados.append({
        'metodo': 'Greedy Natural',
        'colores': metrics_natural['num_colores'],
        'tiempo': t1 - t0,
        'valido': metrics_natural['valido']
    })
    
    order_degree = ordenamiento_grado_desc(G)
    t0 = time.time()
    coloring_degree = greedy_coloring(G, order_degree)
    t1 = time.time()
    metrics_degree = evaluar_coloracion(G, coloring_degree)
    resultados.append({
        'metodo': 'Largest Degree First',
        'colores': metrics_degree['num_colores'],
        'tiempo': t1 - t0,
        'valido': metrics_degree['valido']
    })
    
    order_wp = ordenamiento_welsh_powell(G)
    t0 = time.time()
    coloring_wp = greedy_coloring(G, order_wp)
    t1 = time.time()
    metrics_wp = evaluar_coloracion(G, coloring_wp)
    resultados.append({
        'metodo': 'Welsh-Powell',
        'colores': metrics_wp['num_colores'],
        'tiempo': t1 - t0,
        'valido': metrics_wp['valido']
    })
    
    t0 = time.time()
    coloring_dsatur = dsatur_coloring(G)
    t1 = time.time()
    metrics_dsatur = evaluar_coloracion(G, coloring_dsatur)
    resultados.append({
        'metodo': 'DSATUR',
        'colores': metrics_dsatur['num_colores'],
        'tiempo': t1 - t0,
        'valido': metrics_dsatur['valido']
    })
    
    if model is not None:
        try:
            data = preparar_datos_gnn(G)
            model.eval()
            
            with torch.no_grad():
                scores = model(data.x, data.edge_index)
                ordering = torch.argsort(scores, descending=True).tolist()
            
            t0 = time.time()
            colores_gnn = greedy_coloring_gnn(data.edge_index, data.num_nodes, ordering)
            t1 = time.time()
            
            resultados.append({
                'metodo': 'GNN-guided',
                'colores': colores_gnn,
                'tiempo': t1 - t0,
                'valido': True
            })
        except Exception as e:
            print(f"Error en GNN: {e}")
    
    return resultados

## **4. EVALUACIÓN EN SUITE DE BENCHMARKS**

In [10]:
def evaluar_suite_completa(benchmarks, model=None, repeticiones=5):
    """
    Evalúa todas las heurísticas en una suite de benchmarks.
    """
    resultados_completos = []
    
    for i, (G, nombre) in enumerate(benchmarks):
        print(f"\n[{i+1}/{len(benchmarks)}] Evaluando: {nombre}")
        print(f"  Nodos: {G.number_of_nodes()}, Aristas: {G.number_of_edges()}")
        
        stats = estadisticas_grafo(G, nombre)
        resultados_heuristicas = evaluar_todas_heuristicas(G, model, repeticiones)
        
        for res in resultados_heuristicas:
            resultado_completo = {**stats, **res}
            resultados_completos.append(resultado_completo)
    
    return pd.DataFrame(resultados_completos)

## **5. ANÁLISIS ESTADÍSTICO**

In [11]:
def analisis_estadistico(df):
    """
    Genera análisis estadístico de los resultados.
    """
    print("\n" + "="*80)
    print("ANÁLISIS ESTADÍSTICO POR MÉTODO")
    print("="*80)
    
    resumen = df.groupby('metodo').agg({
        'colores': ['mean', 'std', 'min', 'max'],
        'tiempo': ['mean', 'std'],
        'valido': 'sum'
    }).round(4)
    
    print(resumen)
    
    print("\n" + "="*80)
    print("RANKING DE MÉTODOS (por colores promedio)")
    print("="*80)
    
    ranking = df.groupby('metodo')['colores'].mean().sort_values()
    for i, (metodo, colores) in enumerate(ranking.items(), 1):
        print(f"{i}. {metodo:20s} - {colores:.2f} colores promedio")
    
    return resumen, ranking

## **6. COMPARACIÓN DETALLADA POR GRAFO**

In [12]:
def comparacion_por_grafo(df):
    """
    Muestra comparación detallada por cada grafo.
    """
    print("\n" + "="*80)
    print("COMPARACIÓN DETALLADA POR GRAFO")
    print("="*80)
    
    grafos_unicos = df['nombre'].unique()
    
    for grafo in grafos_unicos:
        df_grafo = df[df['nombre'] == grafo]
        
        print(f"\n{'─'*80}")
        print(f"Grafo: {grafo}")
        
        info = df_grafo.iloc[0]
        print(f"  Nodos: {info['nodos']}, Aristas: {info['aristas']}, "
              f"Densidad: {info['densidad']:.4f}, Grado máx: {info['grado_max']}")
        
        if info['chi_teorico'] is not None:
            print(f"  χ(G) teórico: {info['chi_teorico']}")
        print(f"  Cota superior: Δ+1 = {info['cota_superior']}")
        
        print(f"\n  Resultados por método:")
        
        resumen_metodos = df_grafo.groupby('metodo').agg({
            'colores': ['mean', 'min'],
            'tiempo': 'mean'
        }).round(4)
        
        for metodo in resumen_metodos.index:
            colores_mean = resumen_metodos.loc[metodo, ('colores', 'mean')]
            colores_min = resumen_metodos.loc[metodo, ('colores', 'min')]
            tiempo = resumen_metodos.loc[metodo, ('tiempo', 'mean')]
            print(f"    {metodo:20s}: {colores_mean:6.2f} colores (min: {colores_min:.0f}) - {tiempo:.6f}s")
        
        mejor_metodo = resumen_metodos[('colores', 'mean')].idxmin()
        mejor_resultado = resumen_metodos.loc[mejor_metodo, ('colores', 'mean')]
        print(f"\n  ✓ Mejor método: {mejor_metodo} con {mejor_resultado:.2f} colores")

## **7. VISUALIZACIÓN DE RESULTADOS**

In [13]:
def visualizar_resultados(df):
    """
    Genera visualizaciones de los resultados.
    """
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    resumen_metodos = df.groupby('metodo')['colores'].agg(['mean', 'std'])
    resumen_metodos = resumen_metodos.sort_values('mean')
    
    axes[0, 0].barh(resumen_metodos.index, resumen_metodos['mean'], 
                     xerr=resumen_metodos['std'], capsize=5)
    axes[0, 0].set_xlabel('Número de Colores (promedio)')
    axes[0, 0].set_title('Comparación de Métodos - Colores Usados')
    axes[0, 0].grid(axis='x', alpha=0.3)
    
    tiempo_metodos = df.groupby('metodo')['tiempo'].mean().sort_values()
    axes[0, 1].barh(tiempo_metodos.index, tiempo_metodos.values)
    axes[0, 1].set_xlabel('Tiempo (segundos)')
    axes[0, 1].set_title('Comparación de Métodos - Tiempo de Ejecución')
    axes[0, 1].grid(axis='x', alpha=0.3)
    
    for metodo in df['metodo'].unique():
        df_metodo = df[df['metodo'] == metodo]
        axes[1, 0].scatter(df_metodo['nodos'], df_metodo['colores'], 
                          label=metodo, alpha=0.6)
    axes[1, 0].set_xlabel('Número de Nodos')
    axes[1, 0].set_ylabel('Colores Usados')
    axes[1, 0].set_title('Escalabilidad - Nodos vs Colores')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    for metodo in df['metodo'].unique():
        df_metodo = df[df['metodo'] == metodo]
        axes[1, 1].scatter(df_metodo['densidad'], df_metodo['colores'], 
                          label=metodo, alpha=0.6)
    axes[1, 1].set_xlabel('Densidad del Grafo')
    axes[1, 1].set_ylabel('Colores Usados')
    axes[1, 1].set_title('Densidad vs Colores')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('resultados_evaluacion.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Gráficos guardados en 'resultados_evaluacion.png'")

## **8. ENTRENAMIENTO DE MODELO GNN (OPCIONAL)**

In [ ]:
print("\n" + "#"*80)
print("# ENTRENAMIENTO DE MODELO GNN (OPCIONAL)")
print("#"*80)

print("\n¿Deseas entrenar un modelo GNN para incluirlo en la evaluación?")
print("Esto agregará el método 'GNN-guided' a los resultados.")

# Entrenar modelo GNN en grafos de entrenamiento
entrenar_gnn = True  # Cambiar a False para omitir

if entrenar_gnn:
    print("\n1. Generando grafos de entrenamiento...")
    grafos_entrenamiento = []
    for _ in range(5):
        G_train, _ = generar_benchmark_complejo('scale_free', n=100, m=3)
        grafos_entrenamiento.append(G_train)

    print(f"   {len(grafos_entrenamiento)} grafos generados para entrenamiento")

    print("\n2. Inicializando modelo GNN...")
    # OJO: preparar_datos_gnn() crea X con 3 features (grado, clustering, k-core)
    model_gnn = GNNOrdering(in_channels=3, hidden_channels=32, num_layers=2).to(device)
    optimizer = torch.optim.Adam(model_gnn.parameters(), lr=0.01)

    print("\n3. Entrenando modelo (50 épocas)...")
    model_gnn.train()

    for epoch in range(50):
        total_colors = 0
        for G_train in grafos_entrenamiento:
            data_train = preparar_datos_gnn(G_train)

            optimizer.zero_grad()
            scores = model_gnn(data_train.x, data_train.edge_index)
            ordering = torch.argsort(scores, descending=True).tolist()
            num_colors = greedy_coloring_gnn(data_train.edge_index, data_train.num_nodes, ordering)

            loss = torch.tensor(float(num_colors), dtype=torch.float32, requires_grad=True)
            loss.backward()
            optimizer.step()

            total_colors += num_colors

        if epoch % 10 == 0:
            avg_colors = total_colors / len(grafos_entrenamiento)
            print(f"   Época {epoch:02d}: {avg_colors:.2f} colores promedio")

    print("\n✓ Modelo GNN entrenado exitosamente")
    print("  El modelo será incluido en las evaluaciones con el nombre 'GNN-guided'")
else:
    model_gnn = None
    print("\n⊗ Entrenamiento de GNN omitido")
    print("  Solo se evaluarán las heurísticas clásicas")


################################################################################
# EVALUACIÓN COMPREHENSIVA - NIVEL SIMPLE
################################################################################

Benchmarks cargados: 7

[1/7] Evaluando: Petersen
  Nodos: 10, Aristas: 15

[2/7] Evaluando: Cycle_C20
  Nodos: 20, Aristas: 20

[3/7] Evaluando: Wheel_W15
  Nodos: 15, Aristas: 28

[4/7] Evaluando: Complete_K10
  Nodos: 10, Aristas: 45

[5/7] Evaluando: Bipartite_K10_10
  Nodos: 20, Aristas: 100

[6/7] Evaluando: Tree_n30
  Nodos: 30, Aristas: 29

[7/7] Evaluando: Random_n30_p0.15
  Nodos: 30, Aristas: 75

ANÁLISIS ESTADÍSTICO POR MÉTODO
                     colores                  tiempo         valido
                        mean     std min max    mean     std    sum
metodo                                                             
DSATUR                3.7143  2.8702   2  10  0.0003  0.0002      7
Greedy Natural        4.0000  2.8284   2  10  0.0000  0.0000      7
Largest Deg

## **9. EJECUCIÓN PRINCIPAL - EVALUACIÓN SIMPLE (CON GNN)**

In [ ]:
print("\n" + "#"*80)
print("# EVALUACIÓN COMPREHENSIVA - NIVEL SIMPLE")
print("#"*80)

benchmarks_simple = cargar_benchmark('suite', nivel='simple')
print(f"\nBenchmarks cargados: {len(benchmarks_simple)}")

# Evaluar con o sin GNN según disponibilidad
df_resultados_simple = evaluar_suite_completa(benchmarks_simple, model=model_gnn if entrenar_gnn else None, repeticiones=3)

resumen, ranking = analisis_estadistico(df_resultados_simple)
comparacion_por_grafo(df_resultados_simple)

try:
    visualizar_resultados(df_resultados_simple)
except Exception as e:
    print(f"\nNo se pudieron generar gráficos: {e}")

df_resultados_simple.to_csv('resultados_simple.csv', index=False)
print("\n✓ Resultados guardados en 'resultados_simple.csv'")


################################################################################
# EVALUACIÓN COMPREHENSIVA - NIVEL MEDIO
################################################################################

Benchmarks cargados: 6

[1/6] Evaluando: ScaleFree_n100_m3
  Nodos: 100, Aristas: 291

[2/6] Evaluando: SmallWorld_n100_k6_p0.3
  Nodos: 100, Aristas: 300

[3/6] Evaluando: RandomRegular_n100_d5
  Nodos: 100, Aristas: 250

[4/6] Evaluando: Geometric_n100_r0.15
  Nodos: 100, Aristas: 293

[5/6] Evaluando: PowerlawCluster_n100_m4_p0.3
  Nodos: 100, Aristas: 382

[6/6] Evaluando: Random_n100_p0.1
  Nodos: 100, Aristas: 502

ANÁLISIS ESTADÍSTICO POR MÉTODO
                     colores                  tiempo         valido
                        mean     std min max    mean     std    sum
metodo                                                             
DSATUR                5.0000  1.0954   4   7  0.0053  0.0017      6
Greedy Natural        5.6667  1.5055   4   8  0.0001  0.0000      

## **10. EJECUCIÓN OPCIONAL - EVALUACIÓN MEDIO (CON GNN)**

In [ ]:
print("\n" + "#"*80)
print("# EVALUACIÓN COMPREHENSIVA - NIVEL MEDIO")
print("#"*80)

benchmarks_medio = cargar_benchmark('suite', nivel='medio')
print(f"\nBenchmarks cargados: {len(benchmarks_medio)}")

df_resultados_medio = evaluar_suite_completa(benchmarks_medio, model=model_gnn if entrenar_gnn else None, repeticiones=3)

resumen_medio, ranking_medio = analisis_estadistico(df_resultados_medio)
comparacion_por_grafo(df_resultados_medio)

df_resultados_medio.to_csv('resultados_medio.csv', index=False)
print("\n✓ Resultados guardados en 'resultados_medio.csv'")


################################################################################
# EVALUACIÓN COMPREHENSIVA - NIVEL COMPLEJO
################################################################################

Benchmarks cargados: 5

[1/5] Evaluando: ScaleFree_n500_m5
  Nodos: 500, Aristas: 2475

[2/5] Evaluando: SmallWorld_n500_k8_p0.2
  Nodos: 500, Aristas: 2000

[3/5] Evaluando: RandomRegular_n500_d10
  Nodos: 500, Aristas: 2500

[4/5] Evaluando: Random_n500_p0.05
  Nodos: 500, Aristas: 6149

[5/5] Evaluando: PowerlawCluster_n500_m5_p0.2
  Nodos: 500, Aristas: 2468

ANÁLISIS ESTADÍSTICO POR MÉTODO
                     colores                  tiempo         valido
                        mean     std min max    mean     std    sum
metodo                                                             
DSATUR                6.4000  1.5166   5   9  0.1670  0.0591      5
Greedy Natural        7.8000  2.4900   6  12  0.0005  0.0001      5
Largest Degree First  7.6000  2.0736   6  11  0.0005  

## **11. EJECUCIÓN OPCIONAL - EVALUACIÓN COMPLEJO (CON GNN)**

print("\n" + "#"*80)
print("# EVALUACIÓN COMPREHENSIVA - NIVEL COMPLEJO")
print("#"*80)

benchmarks_complejo = cargar_benchmark('suite', nivel='complejo')
print(f"\nBenchmarks cargados: {len(benchmarks_complejo)}")

df_resultados_complejo = evaluar_suite_completa(
    benchmarks_complejo,
    model=model_gnn if entrenar_gnn else None,
    repeticiones=3
)

resumen_complejo, ranking_complejo = analisis_estadistico(df_resultados_complejo)
comparacion_por_grafo(df_resultados_complejo)

df_resultados_complejo.to_csv('resultados_complejo.csv', index=False)
print("\n✓ Resultados guardados en 'resultados_complejo.csv'")

In [ ]:
print("\n" + "#"*80)
print("# RESUMEN FINAL DE LA EVALUACIÓN")
print("#"*80)

print("\n📊 METODOLOGÍA IMPLEMENTADA:")
print("  ✓ Benchmarks simples: grafos clásicos y pequeños")
print("  ✓ Benchmarks medios: redes sintéticas complejas (100 nodos)")
print("  ✓ Benchmarks complejos: grafos grandes y desafiantes (500 nodos)")
print("  ✓ Soporte para DIMACS y SNAP datasets")

print("\n🔬 HEURÍSTICAS EVALUADAS:")
print("  1. Random (baseline)")
print("  2. Greedy Natural Order")
print("  3. Largest Degree First")
print("  4. Welsh-Powell")
print("  5. DSATUR")
if entrenar_gnn:
    print("  6. GNN-guided ✓ (modelo entrenado incluido)")
else:
    print("  6. GNN-guided (no incluido en esta ejecución)")

print("\n📈 MÉTRICAS ANALIZADAS:")
print("  • Número de colores (promedio, std, min, max)")
print("  • Tiempo de ejecución")
print("  • Validez de la coloración")
print("  • Escalabilidad con tamaño del grafo")
print("  • Comportamiento según densidad")

print("\n💾 ARCHIVOS GENERADOS:")
print("  • resultados_simple.csv")
print("  • resultados_medio.csv (opcional)")
print("  • resultados_complejo.csv (opcional)")
print("  • resultados_evaluacion.png")

if entrenar_gnn:
    print("\n🤖 RESULTADOS GNN:")
    print("  • El modelo GNN fue entrenado y evaluado junto con las heurísticas")
    print("  • Los resultados incluyen la columna 'GNN-guided' en los CSV")
    print("  • Compara el rendimiento de GNN vs métodos clásicos")
    print("  • Para mejorar resultados: entrenar más épocas o con más datos")

print("\n" + "#"*80)

In [ ]:
# (Celda movida a la sección 11). Este bloque ya no es necesario aquí.


################################################################################
# RESUMEN FINAL DE LA EVALUACIÓN
################################################################################

📊 METODOLOGÍA IMPLEMENTADA:
  ✓ Benchmarks simples: grafos clásicos y pequeños
  ✓ Benchmarks medios: redes sintéticas complejas (100 nodos)
  ✓ Benchmarks complejos: grafos grandes y desafiantes (500 nodos)
  ✓ Soporte para DIMACS y SNAP datasets

🔬 HEURÍSTICAS EVALUADAS:
  1. Random (baseline)
  2. Greedy Natural Order
  3. Largest Degree First
  4. Welsh-Powell
  5. DSATUR
  6. GNN-guided (cuando hay modelo entrenado)

📈 MÉTRICAS ANALIZADAS:
  • Número de colores (promedio, std, min, max)
  • Tiempo de ejecución
  • Validez de la coloración
  • Escalabilidad con tamaño del grafo
  • Comportamiento según densidad

💾 ARCHIVOS GENERADOS:
  • resultados_simple.csv
  • resultados_medio.csv (opcional)
  • resultados_complejo.csv (opcional)
  • resultados_evaluacion.png

#########################